## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-29-06-53-00 +0000,wsj,U.S. Treasury Yields Barely Move Ahead of Fed’...,https://www.wsj.com/articles/jgbs-edge-lower-a...
1,2025-10-29-06-52-00 +0000,wsj,Trump Expects to Sign China Trade Deal at Xi S...,https://www.wsj.com/economy/trade/trump-expect...
2,2025-10-29-06-48-28 +0000,nypost,Ex-Army sergeant sentenced for trying to give ...,https://nypost.com/2025/10/29/us-news/ex-army-...
3,2025-10-29-06-48-00 +0000,wsj,"UBS Net Profit Surges on Provision Releases, C...",https://www.wsj.com/business/earnings/ubs-net-...
4,2025-10-29-06-45-43 +0000,bbc,Free baby clothes scheme to support parents fa...,https://www.bbc.com/news/articles/cwyp5dxwne9o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,66
658,new,22
102,melissa,18
101,hurricane,16
12,china,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
162,2025-10-28-20-45-01 +0000,latimes,News Analysis: Trade deal or trade truce? Ques...,https://www.latimes.com/politics/story/2025-10...,129
296,2025-10-28-12-01-13 +0000,wapo,Japan’s new leader tries to charm Trump with g...,https://www.washingtonpost.com/politics/2025/1...,128
353,2025-10-28-07-06-06 +0000,bbc,China controls the rare earths the world buys ...,https://www.bbc.com/news/articles/cj6ny24j0r3o...,123
69,2025-10-29-00-39-18 +0000,cbc,North Korea says it test-fired cruise-missiles...,https://www.cbc.ca/news/world/north-korea-miss...,121
298,2025-10-28-11-49-55 +0000,nypost,Toyota pledges $10B toward new auto plants in ...,https://nypost.com/2025/10/28/us-news/trump-to...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
162,129,2025-10-28-20-45-01 +0000,latimes,News Analysis: Trade deal or trade truce? Ques...,https://www.latimes.com/politics/story/2025-10...
90,74,2025-10-28-23-36-57 +0000,nypost,Hurricane Melissa branded ‘storm of the centur...,https://nypost.com/2025/10/28/world-news/hurri...
238,60,2025-10-28-16-09-00 +0000,wsj,The Republican-led House Oversight Committee i...,https://www.wsj.com/politics/policy/republican...
69,55,2025-10-29-00-39-18 +0000,cbc,North Korea says it test-fired cruise-missiles...,https://www.cbc.ca/news/world/north-korea-miss...
268,51,2025-10-28-13-53-16 +0000,nypost,Ex-Biden press secretary Karine Jean-Pierre’s ...,https://nypost.com/2025/10/28/media/karine-jea...
57,49,2025-10-29-01-40-00 +0000,wsj,Millions of Americans are now learning how muc...,https://www.wsj.com/politics/americans-are-get...
180,48,2025-10-28-19-49-44 +0000,nypost,Disturbing video shows Hamas stage recovery of...,https://nypost.com/2025/10/28/world-news/hamas...
174,47,2025-10-28-20-16-07 +0000,nypost,Chuck Schumer doesn’t need to endorse ‘wack’ M...,https://nypost.com/2025/10/28/us-news/chuck-sc...
134,44,2025-10-28-21-46-55 +0000,wapo,U.S. military strikes four alleged drug boats ...,https://www.washingtonpost.com/national-securi...
103,43,2025-10-28-23-02-55 +0000,nypost,Airport delays are piling up as air traffic co...,https://nypost.com/2025/10/28/us-news/airport-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
